<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Разработка-и-обучение-нескольких-моделей-для-прогноза-модуля-упругости-при-растяжении" data-toc-modified-id="Разработка-и-обучение-нескольких-моделей-для-прогноза-модуля-упругости-при-растяжении-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Разработка и обучение нескольких моделей для прогноза модуля упругости при растяжении</a></span><ul class="toc-item"><li><span><a href="#Прогнозируем-модуль-упругости-при-растяжении" data-toc-modified-id="Прогнозируем-модуль-упругости-при-растяжении-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Прогнозируем модуль упругости при растяжении</a></span></li></ul></li></ul></div>

# Продолжение кода 

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import plotly.express as px
import tensorflow as tf
import sklearn

from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import pickle

from tensorflow import keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation
from pandas import read_excel, DataFrame, Series
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Sequential
from numpy.random import seed
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

In [8]:
df = pd.read_excel('df_clean.xlsx')
df.head()

,Unnamed: 0,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,60.0
1,3,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0,5.0,47.0
2,4,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,57.0
3,5,2.767918,2000.0,748.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,60.0
4,6,2.569620,1910.0,807.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,70.0


In [9]:
#Удаляем первый неинформативный столбец
df.drop(df.columns[df.columns.str.contains('unnamed', case=False)],
           axis=1,
           inplace=True)



In [10]:
df.head()

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1.857143,2030.0,738.736842,50.00,23.750000,284.615385,210.0,70.0,3000.0,220.0,0,4.0,60.0
1,1.857143,2030.0,738.736842,129.00,21.250000,300.000000,210.0,70.0,3000.0,220.0,0,5.0,47.0
2,2.771331,2030.0,753.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,57.0
3,2.767918,2000.0,748.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,60.0
4,2.569620,1910.0,807.000000,111.86,22.267857,284.615385,210.0,70.0,3000.0,220.0,0,5.0,70.0


### Разработка и обучение нескольких моделей для прогноза модуля упругости при растяжении

Сформируем выборки и посмотрим, что получилось

#### Прогнозируем модуль упругости при растяжении

In [11]:
X = df.drop(columns=['Модуль упругости при растяжении, ГПа'],
            axis=1)

In [12]:
X

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
0,1.857143,2030.000000,738.736842,50.000000,23.750000,284.615385,210.000000,3000.000000,220.000000,0,4.000000,60.000000
1,1.857143,2030.000000,738.736842,129.000000,21.250000,300.000000,210.000000,3000.000000,220.000000,0,5.000000,47.000000
2,2.771331,2030.000000,753.000000,111.860000,22.267857,284.615385,210.000000,3000.000000,220.000000,0,5.000000,57.000000
3,2.767918,2000.000000,748.000000,111.860000,22.267857,284.615385,210.000000,3000.000000,220.000000,0,5.000000,60.000000
4,2.569620,1910.000000,807.000000,111.860000,22.267857,284.615385,210.000000,3000.000000,220.000000,0,5.000000,70.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
931,2.271346,1952.087902,912.855545,86.992183,20.123249,324.774576,209.198700,2387.292495,125.007669,1,9.076380,47.019770
932,3.444022,2050.089171,444.732634,145.981978,19.599769,254.215401,350.660830,2360.392784,117.730099,1,10.565614,53.750790
933,3.280604,1972.372865,416.836524,110.533477,23.957502,248.423047,740.142791,2662.906040,236.606764,1,4.161154,67.629684
934,3.705351,2066.799773,741.475517,141.397963,19.246945,275.779840,641.468152,2071.715856,197.126067,1,6.313201,58.261074


In [13]:
y = df['Модуль упругости при растяжении, ГПа']

In [14]:
y

0      70.000000
1      70.000000
2      70.000000
3      70.000000
4      70.000000
         ...    
931    73.090961
932    72.920827
933    74.734344
934    74.042708
935    74.309704
Name: Модуль упругости при растяжении, ГПа, Length: 936, dtype: float64

In [15]:
# разбиваем на тестовую, тренировочную выборки, выделяя предикторы и целевые переменные

X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            test_size=0.3,
                                                            random_state=42)

In [16]:
X_train.head()

,Соотношение матрица-наполнитель,"Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%","Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2","Прочность при растяжении, МПа","Потребление смолы, г/м2",Угол нашивки,Шаг нашивки,Плотность нашивки
133,3.267523,2107.549113,712.148822,60.994261,27.312167,268.485007,121.565802,3460.438212,340.016626,0,4.570892,78.547358
756,2.456261,1984.954030,624.848491,130.912608,23.962165,246.968412,276.827020,2150.828833,126.966998,1,11.704404,53.744193
693,2.548177,1976.868448,630.925295,136.600341,25.027039,280.842690,590.865535,3033.723583,233.705451,1,3.980068,43.107490
853,2.692487,1989.516306,408.332860,38.668500,19.431859,285.364448,80.275335,1811.860024,82.583161,1,8.650571,71.108089
501,3.874909,1968.258174,1120.960568,140.515422,21.039422,236.546673,980.920588,2624.109841,179.673581,1,4.830555,57.205013


In [17]:
y_train

133    81.053293
756    73.137189
693    73.559943
853    75.312292
501    72.843344
         ...    
106    71.680491
270    72.461391
860    75.162208
435    73.119552
102    68.187509
Name: Модуль упругости при растяжении, ГПа, Length: 655, dtype: float64

In [18]:
y_test

321    77.825677
70     69.181599
209    71.337572
656    77.856284
685    70.364622
         ...    
371    69.492433
430    75.280332
292    74.148570
412    69.367179
557    67.803680
Name: Модуль упругости при растяжении, ГПа, Length: 281, dtype: float64

In [19]:
y_train.shape

(655,)

In [20]:
#Функция для сравнения результатов предсказаний с моделью, выдающей среднее значение по тестовой выборке
def mean_model(y_test):
    return [np.mean(y_test) for _ in range(len(y_test))]
y_pred_mean = mean_model(y_test)

Линейная регрессия

In [21]:
#Создадим переменную для названия всех столбцов. Это нам пригодится при построении моделей. И перейдем к визуализации данных
df.columns
#column_names = ["Соотношение матрица-наполнитель","Плотность, кг/м3","модуль упругости, ГПа","Количество отвердителя, м.%",
#         "Содержание эпоксидных групп,%_2","Температура вспышки, С_2","Поверхностная плотность, г/м2",
#         "Модуль упругости при растяжении, ГПа","Прочность при растяжении, МПа","Потребление смолы, г/м2",
#        "Угол нашивки, град","Шаг нашивки","Плотность нашивки"]
column_names = df.columns

Линейная регрессия

In [22]:
#построение модели и визуализация Линейной регрессии
lr = make_pipeline(StandardScaler(),LinearRegression())
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
mae_lr = mean_absolute_error(y_pred_lr, y_test)

print('Linear Regression Results Train:') # Оценка тренировочной выборки
print("Test score: {:.2f}".format(lr.score(X_train, y_train)))
print('Linear Regression Results:')    
print('lr_MAE: ', round(mean_absolute_error(y_test, y_pred_lr)))

print("Test score: {:.2f}".format(lr.score(X_test, y_test))) # Оценка тестовой выборки

Linear Regression Results Train:
Test score: 0.02
Linear Regression Results:
lr_MAE:  2
Test score: -0.01


In [23]:
pickle.dump(lr, open('model_linear.pkl','wb'))

Decision Tree

In [35]:
#Деревья решений - Decision Tree Regressor 

dtr = make_pipeline(StandardScaler(),DecisionTreeRegressor())
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)
mae_dtr = mean_absolute_error(y_pred_dtr, y_test)

print('Decision Tree Regressor Results Train:')
print("Test score: {:.2f}".format(dtr.score(X_train, y_train)))# Оценка тренировочной выборки
print('Decision Tree Regressor Results:')
print('DTR_MAE: ', round(mean_absolute_error(y_test, y_pred_dtr)))
print("Test score: {:.2f}".format(dtr.score(X_test, y_test)))# Оценка тестовой выборки

Decision Tree Regressor Results Train:
Test score: 1.00
Decision Tree Regressor Results:
DTR_MAE:  4
Test score: -1.16


Метод случайного леса - Random Forest Regressor

In [36]:
#построение модели случайный лес
rfr = make_pipeline(StandardScaler(),RandomForestRegressor(n_estimators=15,max_depth=7, random_state=33))
rfr.fit(X_train, y_train.values)
y_pred_forest = rfr.predict(X_test)
mae_rfr = mean_absolute_error(y_pred_forest, y_test)

print('Random Forest Regressor Results Train:')
print("Test score: {:.2f}".format(rfr.score(X_train, y_train))) # Оценка тренировочной выборки
print('Random Forest Regressor Results:')
print('RF_MAE: ', round(mean_absolute_error(y_test, y_pred_forest)))
print("Test score: {:.2f}".format(rfr.score(X_test, y_test))) # Оценка тестовой выборки

Random Forest Regressor Results Train:
Test score: 0.43
Random Forest Regressor Results:
RF_MAE:  2
Test score: -0.04


Метод К ближайших соседей

In [37]:
# Метод К ближайших соседей - K Neighbors Regressor
knn = make_pipeline(StandardScaler(),KNeighborsRegressor(n_neighbors=5))
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
mae_knr = mean_absolute_error(y_pred_knn, y_test)

print('K Neighbors Regressor  Results Train:')
print("Test score: {:.2f}".format(knn.score(X_train, y_train)))# Оценка тренировочной выборки
print('K Neighbors Regressor  Results:')
print('KNN_MAE: ', round(mean_absolute_error(y_test, y_pred_knn)))
print("Test score: {:.2f}".format(knn.score(X_test, y_test)))# Оценка тестовой выборки

K Neighbors Regressor  Results Train:
Test score: 0.16
K Neighbors Regressor  Results:
KNN_MAE:  3
Test score: -0.22


Метода градиентного бустинга - Gradient Boosting Regressor

In [38]:
gbr = make_pipeline(StandardScaler(), GradientBoostingRegressor())
gbr.fit(X_train, y_train)
y_pred_gbr = gbr.predict(X_test)
mae_gbr = mean_absolute_error(y_pred_gbr, y_test)

print('Gradient Boosting Regressor Results Train:')
print("Test score: {:.2f}".format(gbr.score(X_train, y_train))) # Оценка тренировочной выборки
print('Gradient Boosting Regressor Results:')
print('GBR_MAE: ', round(mean_absolute_error(y_test, y_pred_gbr)))

print("Test score: {:.2f}".format(gbr.score(X_test, y_test)))# Оценка тестовой выборки

Gradient Boosting Regressor Results Train:
Test score: 0.46
Gradient Boosting Regressor Results:
GBR_MAE:  2
Test score: -0.09


сравним наши модели по метрике МАЕ

In [39]:
#сравним наши модели по метрике МАЕ
mae_df = {'Регрессор': ['RandomForest', 'Linear Regression', 'KNeighbors', 'DecisionTree', 'Gradient Boosting Regressor'], 'MAE': [mae_rfr, mae_lr, mae_knr, mae_dtr, mae_gbr]} 

mae_df = pd.DataFrame(mae_df)

In [29]:
mae_df

,Регрессор,MAE
0,RandomForest,2.415135
1,Linear Regression,2.414573
2,KNeighbors,2.661645
3,DecisionTree,3.496319
4,Gradient Boosting Regressor,2.493065


Проведем поиск  по сетке гиперпараметров с перекрестной проверкой, количество блоков равно 10 (cv = 10), для 
модели линейной регрессии 

In [40]:
ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=10)
ridge_regressor.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             scoring='neg_mean_squared_error')

In [41]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 100}
-9.683329698909699


In [42]:
prediction_ridge=ridge_regressor.predict(X_test)

In [ ]:
prediction_ridge

In [43]:
print('lr_MAE_ridge: ', round(mean_absolute_error(y_test, prediction_ridge),4))

lr_MAE_ridge:  2.4154
